<a href="https://colab.research.google.com/github/0x444cecilg/data_science_internship/blob/master/no_face_detect_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install facenet_pytorch
import cv2
import os
import time
import urllib.request
import dlib
import numpy as np
import imutils
from skimage import io as img_io
from facenet_pytorch import MTCNN
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, precision_recall_curve, auc, roc_curve

DLIB CNN pretrained face detector

In [ ]:
def detect_faces_DLIB_CNN_eval(images_folder_ref, resize_width=250, thresholds = 0.5):
    def image_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
        # initialize the dimensions of the image to be resized and grab the image size
        dim = None
        (h, w) = image.shape[:2]
        # if both the width and height are None, then return the original image
        if width is None and height is None:
            return image
        # check to see if the width is None
        if width is None:
            # calculate the ratio of the height and construct the dimensions
            r = height / float(h)
            dim = (int(w * r), height)
        # otherwise, the height is None
        else:
            # calculate the ratio of the width and construct the dimensions
            r = width / float(w)
            dim = (width, int(h * r))
        # resize the image
        resized = cv2.resize(image, dim, interpolation=inter)
        # return the resized image
        return resized

    url = "http://arunponnusamy.com/files/mmod_human_face_detector.dat"
    urllib.request.urlretrieve(url, "mmod_human_face_detector.dat")
    cnn_face_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

    users_no_faces = []
    users_faces_detected = []
    y_true = []
    y_pred = []


    start_time = time.time()
    for image_path in os.listdir(images_folder_ref):
        image = cv2.imread(os.path.join(images_folder_ref, image_path))
        if image is None:
            print(f"Warning: Failed to read image '{image_path}'. Skipping.")
            continue
        image = image_resize(image, width=resize_width)
        if "no_face" in image_path:
            ground_truth = 1
        elif "face" in image_path:
            ground_truth = 0
        faces_cnn_raw = cnn_face_detector(image, 1)
        faces_cnn = [face for face in faces_cnn_raw if face.confidence >= thresholds]



        if len(faces_cnn) == 0:
            faces = {}
            for angle in range(0, 360, 90):
                rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                faces_cnn_raw = cnn_face_detector(rotated, 1)
                faces_cnn = [face for face in faces_cnn_raw if face.confidence >= thresholds]
                if len(faces_cnn) > 0:
                    faces[angle] = faces_cnn[0].confidence
            if len(faces) > 0:
                correct_angle = max(faces, key=lambda x: faces[x])
                if faces[correct_angle] < thresholds:
                    users_faces_detected.append(image_path)
                    y_true.append(ground_truth)
                    y_pred.append(0)
                    continue
            users_no_faces.append(image_path)
            y_true.append(ground_truth)
            y_pred.append(1)
        else:
            users_faces_detected.append(image_path)
            y_true.append(ground_truth)
            y_pred.append(0)

    end_time = time.time()
    total_time = end_time - start_time

    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    class_report = classification_report(y_true, y_pred, target_names=["Face Detected", "No Face Detected"], zero_division=1)
    fpr, tpr, _ = roc_curve(y_true, y_pred, pos_label=1)
    roc_auc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)

    return {
        "no_faces_detected": len(users_no_faces),
        "faces_detected": len(users_faces_detected),
        "execution_time_seconds": total_time,
        "confusion_matrix": cm,
        "accuracy": acc,
        "classification_report": class_report,
        "roc_curve": {"fpr": fpr.tolist(), "tpr": tpr.tolist(), "auc": roc_auc},
        "precision_recall_curve": {"precision": precision.tolist(), "recall": recall.tolist(), "auc": pr_auc}
    }

DLIB HOG front face detector + CNN d=face detector

In [ ]:
def detect_faces_DLIB_HOG_CNN_eval(images_folder_ref, resize_width=500, thresholds=0.5):
    def image_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
        # initialize the dimensions of the image to be resized and grab the image size
        dim = None
        (h, w) = image.shape[:2]
        # if both the width and height are None, then return the original image
        if width is None and height is None:
            return image
        # check to see if the width is None
        if width is None:
            # calculate the ratio of the height and construct the dimensions
            r = height / float(h)
            dim = (int(w * r), height)
        # otherwise, the height is None
        else:
            # calculate the ratio of the width and construct the dimensions
            r = width / float(w)
            dim = (width, int(h * r))
        # resize the image
        resized = cv2.resize(image, dim, interpolation=inter)
        # return the resized image
        return resized

    hog_face_detector = dlib.get_frontal_face_detector()

    url = "http://arunponnusamy.com/files/mmod_human_face_detector.dat"
    urllib.request.urlretrieve(url, "mmod_human_face_detector.dat")

    cnn_face_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

    users_no_faces = []
    users_faces_detected = []
    y_true = []
    y_pred = []


    start_time = time.time()
    for image_path in os.listdir(images_folder_ref):
        image = cv2.imread(os.path.join(images_folder_ref, image_path))
        if image is None:
            print(f"Warning: Failed to read image '{image_path}'. Skipping.")
            continue
        image = image_resize(image, width=resize_width)
        if "no_face" in image_path:
            ground_truth = 1
        elif "face" in image_path:
            ground_truth = 0
        faces_hog, detection_confidences, _ = hog_face_detector.run(image, adjust_threshold=thresholds)
        faces_cnn_raw = cnn_face_detector(image, 1)
        faces_cnn = [face for face in faces_cnn_raw if face.confidence >= thresholds]

        if len(faces_hog) == 0 and len(faces_cnn) == 0:
            faces = {}
            for angle in range(0, 360, 90):
                rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                faces_hog = hog_face_detector(rotated)
                faces_cnn_rotated = cnn_face_detector(rotated)
                faces_cnn = [face for face in faces_cnn_rotated if face.confidence >= thresholds]
                if len(faces_hog) > 0 or len(faces_cnn) > 0:
                    if len(faces_cnn) > 0:
                        faces[angle] = faces_cnn[0].confidence
                    else:
                        faces[angle] = 1
                else:
                    faces[angle] = 0
            if len(faces) > 0:
                correct_angle = max(faces, key=lambda x: faces[x])
                if faces[correct_angle] >= thresholds:
                    users_faces_detected.append(image_path)
                    y_true.append(ground_truth)
                    y_pred.append(0)
                    continue
            users_no_faces.append(image_path)
            y_true.append(ground_truth)
            y_pred.append(1)
        else:
            users_faces_detected.append(image_path)
            y_true.append(ground_truth)
            y_pred.append(0)

    end_time = time.time()
    total_time = end_time - start_time

    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    class_report = classification_report(y_true, y_pred, target_names=["Face Detected", "No Face Detected"], zero_division=1)
    fpr, tpr, _ = roc_curve(y_true, y_pred, pos_label=1)
    roc_auc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)

    return {
        "no_faces_detected": len(users_no_faces),
        "faces_detected": len(users_faces_detected),
        "execution_time_seconds": total_time,
        "confusion_matrix": cm,
        "accuracy": acc,
        "classification_report": class_report,
        "roc_curve": {"fpr": fpr.tolist(), "tpr": tpr.tolist(), "auc": roc_auc},
        "precision_recall_curve": {"precision": precision.tolist(), "recall": recall.tolist(), "auc": pr_auc}
    }

MTCNN face detector

In [ ]:
def detect_faces_mtcnn_eval(images_folder_ref, thresholds=0.5, resize_width=250):
    def image_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
      # initialize the dimensions of the image to be resized and grab the image size
      dim = None
      (h, w) = image.shape[:2]
      # if both the width and height are None, then return the original image
      if width is None and height is None:
          return image
      # check to see if the width is None
      if width is None:
          # calculate the ratio of the height and construct the dimensions
          r = height / float(h)
          dim = (int(w * r), height)
      # otherwise, the height is None
      else:
          # calculate the ratio of the width and construct the dimensions
          r = width / float(w)
          dim = (width, int(h * r))
      # resize the image
      resized = cv2.resize(image, dim, interpolation=inter)
      # return the resized image
      return resized

    face_detector_mtcnn = MTCNN(thresholds=(thresholds-0.1, thresholds, thresholds))
    users_no_face = []
    users_with_face = []
    y_true = []
    y_pred = []
    start_time = time.time()

    for image in os.listdir(images_folder_ref):
        image_path = os.path.join(images_folder_ref, image)
        try:
            image_load = img_io.imread(image_path)
        except ValueError:
            print(f"Warning: Unable to read image '{image_path}'. Skipping.")
            continue
        if image_load is None:
            print(f"Warning: Failed to read image '{image_path}'. Skipping.")
            continue
        if "no_face" in image_path:
            ground_truth = 1
        elif "face" in image_path:
            ground_truth = 0
        image_resized = image_resize(image_load, width=resize_width)
        faces_mtcnn = face_detector_mtcnn.detect(image_resized)

        if faces_mtcnn is None or (isinstance(faces_mtcnn, tuple) and faces_mtcnn[0] is None and faces_mtcnn[1] == np.array([None], dtype=object)):
            faces = {}

            for angle in np.arange(0, 360, 90):

                rotated = imutils.rotate(image_resized, angle)
                faces_mtcnn = face_detector_mtcnn.detect(rotated)

                if faces_mtcnn is not None and len(faces_mtcnn) > 0 and isinstance(faces_mtcnn[0], dict):
                    faces[angle] = faces_mtcnn[0]['confidence']
                else:
                    continue
            if len(faces) > 0:
                correct_angle = max(faces, key=lambda x: faces[x])
                if faces[correct_angle] >= thresholds:
                    users_with_face.append(image)
                    y_true.append(ground_truth)
                    y_pred.append(0)
                    continue
            users_no_face.append(image)
            y_true.append(ground_truth)
            y_pred.append(1)
        else:
            users_with_face.append(image)
            y_true.append(ground_truth)
            y_pred.append(0)



    end_time = time.time()
    total_time = end_time - start_time

    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    class_report = classification_report(y_true, y_pred, target_names=["Face Detected", "No Face Detected"], zero_division=1)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)

    results = {
        'no_faces_detected': len(users_no_face),
        'faces_detected': len(users_with_face),
        'execution_time': total_time,
        'confusion_matrix': cm,
        'accuracy': acc,
        'classification_report': class_report,
        'roc_curve': {'fpr': fpr.tolist(), 'tpr': tpr.tolist(), 'auc': roc_auc},
        'precision_recall_curve': {'precision': precision.tolist(), 'recall': recall.tolist(), 'auc': pr_auc}
    }

    return results
